<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Get company posts engagements
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Send_weekly_post_engagement_metrics_by_email.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #posts #interactions #metrics #analytics #automation #naas #hubspot

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

## Input

### Get common variables, functions

In [1]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

### Setup Variables

In [2]:
# Inputs
INPUT_DATABASE = LK_COMPANY_POSTS
OWNER = LK_COMPANY_NAME
OWNER_ID = LK_COMPANY_ID
INPUT_LIKES = LK_COMPANY_POSTS_LIKES
INPUT_COMMENTS = LK_COMPANY_POSTS_COMMENTS
PERIOD = PERIOD_MTD

# Outputs
OUTPUT_DATABASE = LK_COMPANY_ENGAGEMENTS

## Model

### Get your posts

In [3]:
df_posts = get_data(INPUT_DATABASE)
print("✅ Posts fetched:", len(df_posts))
df_posts.head(1)

✅ Posts fetched: 126


,ACTIVITY_ID,PUBLISHED_DATE,AUTHOR_NAME,AUTHOR_URL,SUBDESCRIPTION,TITLE,TEXT,CHARACTER_COUNT,TAGS,TAGS_COUNT,...,POLL_ID,POLL_QUESTION,POLL_RESULTS,POST_URL,VIEWS,COMMENTS,LIKES,SHARES,ENGAGEMENT_SCORE,DATE_EXTRACT
0,6946860124556681216,2022-06-26 16:22:00+02:00,naas.ai,https://www.linkedin.com/company/70506391,23 hours ago,No comment. 👌,No comment. 👌,13,NaN,0,...,NaN,NaN,NaN,https://www.linkedin.com/feed/update/urn:li:ac...,221,0,2,0,0.00905,2022-06-27 18:21:50


### Get who likes your posts

In [4]:
df_likes = get_data(INPUT_LIKES)
print("✅ Likes fetched:", len(df_likes))
df_likes.head(1)

[Errno 2] No such file or directory: '/home/ftp/Naas Content Engine/LinkedIn/Inputs/LINKEDIN_COMPANY_POSTS_LIKES_70506391.csv'
✅ Likes fetched: 0


""


### Get who comments your posts

In [5]:
df_comments = get_data(INPUT_COMMENTS)
print("✅ Comments fetched:", len(df_comments))
df_comments.head(1)

[Errno 2] No such file or directory: '/home/ftp/Naas Content Engine/LinkedIn/Inputs/LINKEDIN_COMPANY_POSTS_COMMENTS_70506391.csv'
✅ Comments fetched: 0


""


## Output

In [6]:
# Get interactions
def get_interactions(interaction, post_url):
    df = pd.DataFrame()
    try:
        if interaction == "LIKES":
            df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
        elif interaction == "COMMENTS":
            df = linkedin.connect(LI_AT, JSESSIONID).post.get_comments(post_url)
    except Exception as e:
        if e.response.status_code:
            print(e)
    return df

In [8]:
def update_interactions(df_posts,
                        df_interaction,
                        interaction,
                        csv_output,
                        no_posts=10,
                        min_updated_time=300):
    # Init
    df_out = df_interaction.copy()
    
    # Get all interactions if dataframe init empty or not complete
    if len(df_posts) == 0:
        return pd.DataFrame()

    if len(df_out) > 0:
        if "DATE_EXTRACT" in df_out.columns:
            last_update_date = df_out["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                df_posts = df_posts[:no_posts]
            else:
                print(f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago.")
                return df_out.reset_index(drop=True)
    else:
        df_posts["SCENARIO"] = pd.to_datetime(df_posts["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
        df_posts = df_posts[df_posts["SCENARIO"] == datetime.now().strftime(PERIOD)].reset_index(drop=True)
        print("Posts this month:", len(df_posts))
        
    # Loop on posts
    for index, row in df_posts.iterrows():
        df_update = pd.DataFrame()
        post_title = row.TITLE
        post_author = row.AUTHOR_NAME
        post_url = row.POST_URL
        post_date = row.PUBLISHED_DATE
        count_interactions = row[interaction]
        print(f"🔄 {index+1} - Update started on: '{post_title}' ({post_url})")
        
        # Get interactions from post URL
        if len(df_interaction) > 0:
            tmp_df = df_interaction[df_interaction.POST_URL == post_url]
            no_interactions = len(tmp_df)
            if count_interactions != no_interactions:
                print(f"--> {count_interactions} post interaction count vs {no_interactions} interactions.")
                df_update = get_interactions(interaction, post_url)
            else:
                print("--->🛑 Nothing to update.")
        else:
            df_update = get_interactions(interaction, post_url)
        
        # Concat dataframe and save dataframe in CSV
        if len(df_update) > 0:
            print(f"---> {len(df_update)} interactions fetched.")
            df_update['TITLE'] = post_title
            df_update['AUTHOR_NAME'] = post_author
            df_update['PUBLISHED_DATE'] = post_date
            keys = ["POST_URL", "PROFILE_ID"]
            if interaction == "COMMENTS":
                keys = ["POST_URL", "PROFILE_ID", "CREATED_TIME"]
            df_out = pd.concat([df_update, df_out]).drop_duplicates(keys, keep="first")
            output_path = save_data(df_out, csv_output)
            
    # Add dependency in production
    print(f"✅ {len(df_out)} '{interaction}' fetched.")
    # Return all interactions
    return df_out.reset_index(drop=True)

### Update likes
It will update your last 10 posts like's from LinkedIn API.<br>
PS: On the first execution all posts like's will be retrieved.

In [9]:
df_update_likes = update_interactions(df_posts,
                                      df_likes,
                                      "LIKES",
                                      INPUT_LIKES)
df_update_likes.head(1)

Posts this month: 16
🔄 1 - Update started on: 'No comment. 👌' (https://www.linkedin.com/feed/update/urn:li:activity:6946860124556681216)
---> 2 interactions fetched.
👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved in CSV: LinkedIn/Inputs/LINKEDIN_COMPANY_POSTS_LIKES_70506391.csv
🔄 2 - Update started on: 'Most of the people working in data think notebooks are only made for experimentation and convey bad practices, but it's more a question of how you use them.' (https://www.linkedin.com/feed/update/urn:li:activity:6945433259858132992)
🔄 3 - Update started on: 'Want to use AI to label the sentiment of your comments?' (https://www.linkedin.com/feed/update/urn:li:activity:6944613208842227712)
---> 11 interactions fetched.
👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved i

,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,BACKGROUND_PICTURE,PROFILE_TYPE,REACTION_TYPE,POST_URL,DATE_EXTRACT,TITLE,AUTHOR_NAME,PUBLISHED_DATE
0,ACoAAC5LlP0BtWg2TdS_5IspIp4XYk2X5f1zAtI,https://www.linkedin.com/in/ACoAAC5LlP0BtWg2Td...,sriniketh-jayasendil,Sriniketh,Jayasendil,Sriniketh Jayasendil,DS & ML Enthusiast | Learning Web Dev | Open S...,https://media-exp2.licdn.com/dms/image/C4E03AQ...,https://media-exp2.licdn.com/dms/image/C4E16AQ...,PROFILE,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...,2022-06-28 21:23:28,Build your company metrics database with naas....,naas.ai,2022-06-02 12:53:31+02:00


### Update comments
It will update your last 10 posts comment's from LinkedIn API.<br>
PS: On the first execution all posts comment's will be retrieved.

In [10]:
df_update_comments = update_interactions(df_posts,
                                         df_comments,
                                         "COMMENTS",
                                         INPUT_COMMENTS)
df_update_comments.head(1)

Posts this month: 16
🔄 1 - Update started on: 'No comment. 👌' (https://www.linkedin.com/feed/update/urn:li:activity:6946860124556681216)
🔄 2 - Update started on: 'Most of the people working in data think notebooks are only made for experimentation and convey bad practices, but it's more a question of how you use them.' (https://www.linkedin.com/feed/update/urn:li:activity:6945433259858132992)
🔄 3 - Update started on: 'Want to use AI to label the sentiment of your comments?' (https://www.linkedin.com/feed/update/urn:li:activity:6944613208842227712)
---> 2 interactions fetched.
👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved in CSV: LinkedIn/Inputs/LINKEDIN_COMPANY_POSTS_COMMENTS_70506391.csv
🔄 4 - Update started on: 'Wondering how to read a dataframe from your files stored in Amazon Web Services (AWS) S3 ? ' (https://www.linkedin.com/feed/update/urn:li:activity:694318528829476

,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,BACKGROUND_PICTURE,PROFILE_TYPE,...,CREATED_TIME,LANGUAGE,DISTANCE,COMMENTS,LIKES,POST_URL,DATE_EXTRACT,TITLE,AUTHOR_NAME,PUBLISHED_DATE
0,70506391,https://www.linkedin.com/company/70506391,naas-ai,None,None,naas.ai,None,https://media-exp2.licdn.com/dms/image/C560BAQ...,None,COMPANY,...,2022-06-16 12:05:26,English,None,0,0,https://www.linkedin.com/feed/update/urn:li:ac...,2022-06-28 21:24:37,Dashboards do not need to be perfect the first...,naas.ai,2022-06-14 12:54:42+02:00


### Create interactions database
- Concat LIKES and COMMENTS in single database
- Create note for HubSpot

In [11]:
def create_interactions_db(df_likes, df_comments):
    # Init outputs
    df = pd.DataFrame()
    
    # Dataframe likes
    df_likes["REACTION"] = "LIKES"

    # Dataframe comments
    df_comments["REACTION"] = "COMMENTS"
    
    # Concat
    df = pd.concat([df_likes, df_comments]).fillna("Not defined").sort_values(by="PUBLISHED_DATE", ascending=False)
    
    # Cleaning
    to_keep = [
        "PROFILE_ID",
        "PROFILE_URL",
        "PUBLIC_ID",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "REACTION",
        "TEXT",
        "TITLE",
        "PUBLISHED_DATE",
        "AUTHOR_NAME",
        "POST_URL",
    ]
    df = df[to_keep]
    
    print(f"✅ {len(df)} interactions fetched.")
    return df.reset_index(drop=True)

df_interactions = create_interactions_db(df_update_likes, df_update_comments)
df_interactions.head(3)

✅ 70 interactions fetched.


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,REACTION,TEXT,TITLE,PUBLISHED_DATE,AUTHOR_NAME,POST_URL
0,ACoAABYVsxEBFJ5cAeOCukhSw2dAK8h_pMtwg1E,https://www.linkedin.com/in/ACoAABYVsxEBFJ5cAe...,valentin-piquard-955105a4,Valentin,Piquard,Valentin Piquard,Product-Led Growth @Naas.ai | Building an Open...,LIKES,Not defined,No comment. 👌,2022-06-26 16:22:00+02:00,naas.ai,https://www.linkedin.com/feed/update/urn:li:ac...
1,ACoAABRK0_8BfD3WlBxZX0cc7IJj7GfYW087PME,https://www.linkedin.com/in/ACoAABRK0_8BfD3WlB...,hossam-medhat,Hossam,Medhat,Hossam Medhat,Associate Product Manager | Business Analyst |...,LIKES,Not defined,No comment. 👌,2022-06-26 16:22:00+02:00,naas.ai,https://www.linkedin.com/feed/update/urn:li:ac...
2,ACoAAABbOD8BF_9zDdGmvl0Tu16BnKqThxQEgik,https://www.linkedin.com/in/ACoAAABbOD8BF_9zDd...,njgroene,Nikolaj,Groeneweg,Nikolaj Groeneweg,Data & AI · La French Tech 🚀,COMMENTS,Thanks for the share 🤗,Want to use AI to label the sentiment of your ...,2022-06-20 11:33:33+02:00,naas.ai,https://www.linkedin.com/feed/update/urn:li:ac...


### Save data

In [12]:
save_data(df_interactions, OUTPUT_DATABASE)

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved in CSV: LinkedIn/Inputs/LINKEDIN_COMPANY_ENGAGEMENTS_70506391.csv
